# Datafarm client usage examples

In [2]:
import datetime
import os
import json

import pandas as pd
from requests import HTTPError
from dotenv import load_dotenv
from datafarmclient.base import DatafarmClient

load_dotenv()
api_url = os.getenv("DATAFARM_API_URL")
api_key = os.getenv("DATAFARM_API_KEY")
assert api_url is not None
assert api_key is not None
repo = DatafarmClient(api_url=api_url)
repo.login(api_key=api_key)

True

## Variables
You can access and set variables in Datafarm using the client.


### Get and set variables
You can specify a variable name and value to set a variable in Datafarm.

In [ ]:
res = repo.set_variable(name="olsen", value=4)
repo.get_variable(name="olsen")

In [ ]:
res = repo.set_variable(name="olsen", value="hello")
repo.get_variable(name="olsen")

In [ ]:
res = repo.set_variable(name="olsen", value=False)
repo.get_variable(name="olsen")

In [ ]:
res = repo.set_variable(name="olsen", value=datetime.datetime.now())
repo.get_variable(name="olsen")

The type of the variable is inferred from the value or can optionally be specified in the `variable_type` argument.

In [ ]:
res = repo.set_variable(name="olsen", value="20200101", variable_type="datetime")
repo.get_variable(name="olsen")

### Object variables
You can also assing objects (files) to variables using `get_object_variable` and `set_object_variable`.

In [ ]:
res = repo.set_object_variable(name="olsen", file_path="../tests/data/test_upload.txt")
repo.get_object_variable(name="olsen")

In [ ]:
path = repo.get_object_variable(name="olsen", output_dir=".")
print(path)
with open(path, "r") as f:
    print(f.read())
os.remove(path)

In [ ]:
path = repo.get_object_variable(name="olsen", output_path="custom_path.txt")
print(path)
with open(path, "r") as f:
    print(f.read())
os.remove(path)

In [ ]:
res = repo.set_object_variable(name="olsen", file_path="../tests/data/test_upload.png")
path = repo.get_object_variable(name="olsen", output_dir=".")
print(path)
# with open(path, 'r') as f:
#     print(f.read())
# os.remove(path)

### Create Entity

In [ ]:
from datafarmclient.exceptions import EntityExistsError


try:
    res = repo.entities.create(
        "enTimeSeries", id_name="test_ts_1", fields={"IDDescription": "test1"}
    )
except EntityExistsError:
    print("Entity already exists")

In [ ]:
entity_id = repo.get_entity_id("enTimeSeries", "test_ts_1")
print(entity_id)

In [ ]:
res = repo.update_entity(
    "enTimeSeries", entity_id="test_ts_1", fields={"IDDescription": "test1-updated"}
)
print(res.json())
repo.list_time_series().tail(1)

In [ ]:
res = repo.delete_entity(class_id="enTimeSeries", entity_id=entity_id)
print(res.json())

In [ ]:
repo.list_time_series().tail(1)

### Show statistics

In [ ]:
time_series_list = repo.list_time_series()
time_series_list.tail(10)

In [ ]:
repo.get_statistics(time_series_list["EntityID"].iloc[-10:])

### Get time series data

In [ ]:
data = repo.timeseries.get(
    time_series_id="testapi.insert", ascending=True, limit=10, fields=["vtDateTime"]
)
data

### Insert time series data

Here we prepare some data to insert in form of a dataframe.

In [ ]:
import random

n = 5
timestamps = [pd.Timestamp("1900-01-01") + pd.Timedelta(days=i) for i in range(n)]
insert_data = pd.DataFrame(
    {
        "TimeStamp": timestamps,
        "Data": [random.random() for _ in range(n)],
        "Quality": ["ok"] * n,
    }
)
insert_data.loc[0, "Data"] = None
insert_data

To insert the data we use the `insert_data` method:

In [ ]:
try:
    res = repo.insert_data("testapi.insert", insert_data, bulk_insert=True)
except HTTPError as e:
    print(e)

In [ ]:
res.json()

In [ ]:
data = repo.get_data("testapi.insert")
data.head(10)

We can peek at the json body for the call to the API. The NaN value is represented as {"N": 1, "V": 0}.

In [ ]:
json.loads(res.request.body)

### Update qualities

In [ ]:
qualities = list(repo.qualities["EntityID"])
print(qualities)

In [ ]:
repo.update_data_quality(
    "testapi.insert", timestamps=timestamps, quality=qualities[: len(timestamps)]
)
data = repo.get_data("testapi.insert", limit=10)
data

### Delete data

Either delete in range or delete specific values.

In [ ]:
start, end = timestamps[:2]
print(f"Deleting data in range [{start}, {end})\n")
res = repo.delete_data("testapi.insert", start=start, end=end)
data = repo.get_data("testapi.insert", limit=10)
data

In [ ]:
timestamps = data.index[:3]
repo.delete_data("testapi.insert", timestamps=timestamps)
data = repo.get_data("testapi.insert", limit=10)
data

### Using the context manager

In [ ]:
with DatafarmClient(api_key) as repo:
    repo.login(api_key=api_key)
    data = repo.get_data(time_series_id="testapi.insert", limit=2)
data